In [1]:
RUN_ID = '4c379b0437794f32bbe728c182db57c6'

In [2]:
print(f'{RUN_ID=}')

RUN_ID='4c379b0437794f32bbe728c182db57c6'


In [3]:
if RUN_ID is None:
    print('Error! provide RUN_ID parameter')

In [4]:
import mlflow
import os
import sys
ml_flow_url = os.environ.get('MLFLOW_URL', "http://192.168.10.38:5000")
mlflow.set_tracking_uri(ml_flow_url)
mlflow.set_experiment("Обучение анализатора")
 

<Experiment: artifact_location='mlflow-artifacts:/734503408361585659', creation_time=1678465756231, experiment_id='734503408361585659', last_update_time=1678465756231, lifecycle_stage='active', name='Обучение анализатора', tags={}>

In [5]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
import analyser.hyperparams 

Nemoware Analyser v23.3.15
USING WORKDIR: [/root/artem/work]
 set ENV GPN_WORK_DIR to override
datasets_dir: [/root/artem/work/datasets]
reports_dir: [/root/artem/work/reports]


/root/artem/analyser/analyser/hyperparams.py:15: UserWarning: please set GPN_WORK_DIR environment variable
  warnings.warn('please set GPN_WORK_DIR environment variable')


In [6]:
mlflow_client = mlflow.tracking.MlflowClient(tracking_uri=ml_flow_url)
mlflow_client

In [7]:

tmp_path = mlflow_client.download_artifacts(run_id =RUN_ID, path='' )

/tmp/ipykernel_2349567/1970305853.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  tmp_path = mlflow_client.download_artifacts(run_id =RUN_ID, path='' )


In [8]:
from tf_support.super_contract_model import make_att_model
umodel=make_att_model()

2023-03-15 15:23:48.515543: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[documents.py:270 -             __init__() ] loading word cases stats model from: /root/artem/analyser/analyser/vocab/word_cases_stats.pickle
[tools.py:29 -             __init__() ] model_checkpoint_path: /root/artem/analyser/analyser/vocab
2023-03-15 15:23:49.367695: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-03-15 15:23:49.389561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-15 15:23:49.390258: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB device

In [9]:
from pathlib import Path
umodel.load_weights(Path(tmp_path) /  f'{umodel.name}.h5')
umodel.summary()

Model: "make_att_model_03"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text_emb (InputLayer)     [(None, None, 1024)] 0                                            
__________________________________________________________________________________________________
token_features (InputLayer)     [(None, None, 15)]   0                                            
__________________________________________________________________________________________________
input_text_emb_norm (LayerNorma (None, None, 1024)   2048        input_text_emb[0][0]             
__________________________________________________________________________________________________
token_features_norm (LayerNorma (None, None, 15)     30          token_features[0][0]             
__________________________________________________________________________________

In [10]:
umodel.save_weights(Path(analyser.hyperparams.models_path) / (umodel.name + ".h5"))